<a href="https://colab.research.google.com/github/alifarah94/DS-Unit-2-Linear-Models/blob/master/Week_5_Day_3_regression_classification_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

Instead, predict property sales prices for **One Family Dwellings** (`BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'`). 

Use a subset of the data where the **sale price was more than \\$100 thousand and less than $2 million.** 

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.

- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Fit a ridge regression model with multiple features.
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.


## Stretch Goals
- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [130]:
#df = df.dropna(axis=0)

BOROUGH                               0
NEIGHBORHOOD                          0
BUILDING_CLASS_CATEGORY               0
TAX_CLASS_AT_PRESENT                  1
BLOCK                                 0
LOT                                   0
BUILDING_CLASS_AT_PRESENT             1
ADDRESS                               0
APARTMENT_NUMBER                  17839
ZIP_CODE                              1
RESIDENTIAL_UNITS                     1
COMMERCIAL_UNITS                      1
TOTAL_UNITS                           1
LAND_SQUARE_FEET                     53
GROSS_SQUARE_FEET                     1
YEAR_BUILT                           35
TAX_CLASS_AT_TIME_OF_SALE             0
BUILDING_CLASS_AT_TIME_OF_SALE        0
SALE_PRICE                            0
SALE_DATE                             0
dtype: int64

In [0]:
df = df.drop('EASE-MENT',axis=1)

In [0]:
df['SALE_DATE'] = df['SALE_DATE'].fillna(df['SALE_DATE'].mean()) # replace missing years by mean

In [53]:
df.select_dtypes(exclude='number').describe().T

,count,unique,top,freq
BOROUGH,23040,5,4,7494
NEIGHBORHOOD,23040,11,OTHER,19117
BUILDING_CLASS_CATEGORY,23040,44,01 ONE FAMILY DWELLINGS,5061
TAX_CLASS_AT_PRESENT,23039,10,1,11071
BUILDING_CLASS_AT_PRESENT,23039,146,D4,3408
ADDRESS,23040,22691,100 JEROME STREET,6
APARTMENT_NUMBER,5201,1724,4,99
LAND_SQUARE_FEET,22987,3652,0,7500
BUILDING_CLASS_AT_TIME_OF_SALE,23040,147,D4,3408
SALE_DATE,23040,120,01/24/2019,480


In [54]:
df['BOROUGH'].value_counts()

4    7494
3    6647
1    4803
2    2188
5    1908
Name: BOROUGH, dtype: int64

In [0]:
target = 'SALE_PRICE'
high_cardinality = ['NEIGHBORHOOD','BUILDING_CLASS_CATEGORY','TAX_CLASS_AT_PRESENT','BUILDING_CLASS_AT_PRESENT','ADDRESS','APARTMENT_NUMBER','LAND_SQUARE_FEET','BUILDING_CLASS_AT_TIME_OF_SALE','SALE_DATE'] # removing these features from dataframe
features = df.columns.drop([target]+high_cardinality)

In [0]:
df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'], infer_datetime_format=True)

In [0]:
cutoff = pd.to_datetime('2019-04-01')

In [0]:
df_train = df[df.SALE_DATE<cutoff]
df_test = df[df.SALE_DATE>=cutoff]

In [0]:
x_train = df_train[features]
y_train = df_train[[target]]
x_test = df_test[features]
y_test = df_test[target]

In [115]:
x_train.shape

(18167, 10)

In [0]:
# this cell one hot encodes x_train and x_test

import category_encoders as ce
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge # ridge regression
from sklearn.metrics import mean_absolute_error




In [0]:

# first one encoding the data
encoder = ce.OneHotEncoder(use_cat_names=True)

x_train = encoder.fit_transform(x_train)
x_test = encoder.transform(x_test)


In [0]:
x_train = x_train.dropna(axis=1) # removing nan values from train and test
x_test = x_test.dropna(axis=1)

In [0]:
# time to pick features to use

from sklearn.feature_selection import f_regression, SelectKBest # import selectKbest

selector = SelectKBest(score_func=f_regression,k=5) # pick the top 5 features



In [250]:
# top 5 features in the datframe for both test and train
x_train_top_5 = selector.fit_transform(x_train,y_train) 
x_test_top_5 = selector.fit_transform(x_test,y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
# time to scale both test and train

scaler = StandardScaler()

x_train_scal = scaler.fit_transform(x_train_top_5)
x_test_scal = scaler.fit_transform(x_test_top_5)

In [0]:
x_train_scal = pd.DataFrame(x_train_scal)
x_test_scal = pd.DataFrame(x_test_scal)

In [184]:
x_test_scal.isna().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
13    0
dtype: int64

In [253]:
x_test_top_5

array([[   1,    0,  695, 1562,    2],
       [   1,    0,  767, 1609,    2],
       [   1,    0,  767, 1612,    2],
       ...,
       [   0,    1,  131,    4,    2],
       [   0,    1, 8932,   18,    1],
       [   0,    1, 1216, 1161,    2]])

In [0]:
# create ridge regression model

model = Ridge(alpha=1)
model.fit(x_train_scal,y_train)
y_pred = model.predict(x_test_scal)

In [0]:
#calculate mean absolute error

model_MAE = mean_absolute_error(y_test,y_pred)

In [256]:
model_MAE

7625280.818534474